In [1]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
from model import Fasttext
from data import AG_Data
import numpy as np
from tqdm import tqdm

In [2]:
import config as argumentparser
config = argumentparser.ArgumentParser()

In [3]:
if config.cuda and torch.cuda.is_available():  # 是否使用gpu
    torch.cuda.set_device(config.gpu)

In [4]:
torch.cuda.is_available() # 查看gpu是否可用

False

In [5]:
# 导入训练集
training_set = AG_Data("/AG/train.csv",min_count=config.min_count,
                       max_length=config.max_length,n_gram=config.n_gram)
training_iter = torch.utils.data.DataLoader(dataset=training_set,
                                            batch_size=config.batch_size,
                                            shuffle=True,
                                            num_workers=0)

In [6]:
# 导入测试集
test_set = AG_Data(data_path="/AG/test.csv",min_count=config.min_count,
                   max_length=config.max_length,n_gram=config.n_gram,word2id=training_set.word2id,
                   uniwords_num=training_set.uniwords_num)
test_iter = torch.utils.data.DataLoader(dataset=test_set,
                                        batch_size=config.batch_size,
                                        shuffle=False,
                                        num_workers=0)

In [7]:
model = Fasttext(vocab_size=training_set.uniwords_num+100000,embedding_size=config.embed_size,
                 max_length=config.max_length,label_num=config.label_num) # 初始化模型

In [8]:
if config.cuda and torch.cuda.is_available(): # 如果使用gpu，将模型送进gpu
    model.cuda()

In [9]:
criterion = nn.CrossEntropyLoss() # 构建loss结构
optimizer = optim.Adam(model.parameters(), lr=config.learning_rate) #构建优化器
loss  = -1

In [15]:
def get_test_result(data_iter,data_set):
    # 生成测试结果
    model.eval()
    true_sample_num = 0
    for data, label in data_iter:
        if config.cuda and torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        else:
            data = torch.autograd.Variable(data).long()
        out = model(data)
        true_sample_num += np.sum((torch.argmax(out, 1) == label.long()).cpu().numpy())
    acc = true_sample_num / data_set.__len__()
    return acc

In [16]:
for epoch in range(1):
    model.train()
    process_bar = tqdm(training_iter)
    for data, label in process_bar:
        if config.cuda and torch.cuda.is_available():
            data = data.cuda()
            label = label.cuda()
        else:
            data = torch.autograd.Variable(data).long()
        label = torch.autograd.Variable(label).squeeze()
        out = model(data)
        loss_now = criterion(out, autograd.Variable(label.long()))
        if loss == -1:
            loss = loss_now.data.item()
        else:
            loss = 0.95*loss+0.05*loss_now.data.item()
        process_bar.set_postfix(loss=loss_now.data.item())
        process_bar.update()
        optimizer.zero_grad()
        loss_now.backward()
        optimizer.step()
    test_acc = get_test_result(test_iter, test_set)
    print("The test acc is: %.5f" % test_acc)




  0%|                                                                                         | 0/1875 [00:00<?, ?it/s]


  0%|                                                                            | 0/1875 [00:00<?, ?it/s, loss=0.0353]


  0%|                                                                     | 1/1875 [00:00<01:26, 21.68it/s, loss=0.132]


  0%|                                                                    | 2/1875 [00:00<01:16, 24.63it/s, loss=0.0679]


  0%|                                                                    | 3/1875 [00:00<01:10, 26.71it/s, loss=0.0679]


  0%|                                                                    | 3/1875 [00:00<01:10, 26.71it/s, loss=0.0727]


  0%|▏                                                                   | 4/1875 [00:00<01:10, 26.71it/s, loss=0.0889]


  0%|▏                                                                   | 5/1875 [00:00<01:10, 26.71it/s, loss=0.0514]


  0%|▏               

  3%|█▊                                                                  | 49/1875 [00:01<01:02, 29.34it/s, loss=0.158]


  3%|█▊                                                                  | 50/1875 [00:01<01:02, 29.34it/s, loss=0.132]


  3%|█▊                                                                  | 51/1875 [00:01<00:58, 31.42it/s, loss=0.132]


  3%|█▊                                                                  | 51/1875 [00:01<01:02, 29.34it/s, loss=0.132]


  3%|█▊                                                                  | 51/1875 [00:01<01:02, 29.34it/s, loss=0.161]


  3%|█▊                                                                 | 52/1875 [00:01<01:02, 29.34it/s, loss=0.0817]


  3%|█▉                                                                 | 53/1875 [00:01<01:02, 29.34it/s, loss=0.0824]


  3%|█▉                                                                 | 54/1875 [00:01<01:02, 29.34it/s, loss=0.0894]


  3%|█▉                 

  5%|███▍                                                               | 97/1875 [00:03<00:55, 31.87it/s, loss=0.0913]


  5%|███▌                                                                | 98/1875 [00:03<00:55, 31.87it/s, loss=0.126]


  5%|███▌                                                                | 99/1875 [00:03<00:55, 32.04it/s, loss=0.126]


  5%|███▌                                                               | 99/1875 [00:03<00:55, 32.04it/s, loss=0.0698]


  5%|███▌                                                              | 100/1875 [00:03<00:55, 32.04it/s, loss=0.0483]


  5%|███▌                                                               | 101/1875 [00:03<00:55, 32.04it/s, loss=0.122]


  5%|███▌                                                              | 102/1875 [00:03<00:55, 32.04it/s, loss=0.0835]


  5%|███▋                                                              | 103/1875 [00:03<00:55, 31.77it/s, loss=0.0835]


  5%|███▋               

  8%|█████                                                             | 144/1875 [00:04<00:57, 30.28it/s, loss=0.0446]


  8%|█████▏                                                             | 145/1875 [00:04<00:57, 30.28it/s, loss=0.154]


  8%|█████▏                                                             | 146/1875 [00:04<00:57, 30.28it/s, loss=0.111]


  8%|█████▎                                                             | 147/1875 [00:04<00:53, 32.40it/s, loss=0.111]


  8%|█████▎                                                             | 147/1875 [00:04<00:57, 30.27it/s, loss=0.111]


  8%|█████▏                                                            | 147/1875 [00:04<00:57, 30.27it/s, loss=0.0317]


  8%|█████▎                                                             | 148/1875 [00:04<00:57, 30.27it/s, loss=0.094]


  8%|█████▎                                                             | 149/1875 [00:04<00:57, 30.27it/s, loss=0.112]


  8%|█████▎             

 10%|██████▊                                                           | 192/1875 [00:06<00:53, 31.64it/s, loss=0.0696]


 10%|██████▉                                                             | 193/1875 [00:06<00:53, 31.64it/s, loss=0.05]


 10%|██████▊                                                           | 194/1875 [00:06<00:53, 31.64it/s, loss=0.0247]


 10%|██████▊                                                           | 195/1875 [00:06<00:50, 33.56it/s, loss=0.0247]


 10%|██████▊                                                           | 195/1875 [00:06<00:53, 31.27it/s, loss=0.0247]


 10%|██████▊                                                           | 195/1875 [00:06<00:53, 31.27it/s, loss=0.0806]


 10%|███████                                                            | 196/1875 [00:06<00:53, 31.27it/s, loss=0.104]


 11%|██████▉                                                           | 197/1875 [00:06<00:53, 31.27it/s, loss=0.0455]


 11%|██████▉            

...

 90%|████████████████████████████████████████████████████████████▎      | 1688/1875 [00:54<00:05, 32.70it/s, loss=0.12]


 90%|███████████████████████████████████████████████████████████▍      | 1689/1875 [00:54<00:05, 32.70it/s, loss=0.116]


 90%|███████████████████████████████████████████████████████████▍      | 1690/1875 [00:54<00:05, 32.54it/s, loss=0.116]


 90%|███████████████████████████████████████████████████████████▍      | 1690/1875 [00:54<00:05, 32.54it/s, loss=0.128]


 90%|██████████████████████████████████████████████████████████▌      | 1691/1875 [00:54<00:05, 32.54it/s, loss=0.0933]


 90%|██████████████████████████████████████████████████████████▋      | 1692/1875 [00:54<00:05, 32.54it/s, loss=0.0651]


 90%|██████████████████████████████████████████████████████████▋      | 1693/1875 [00:54<00:05, 32.54it/s, loss=0.0579]


 90%|██████████████████████████████████████████████████████████▋      | 1694/1875 [00:54<00:05, 32.75it/s, loss=0.0579]


 90%|███████████████████

 93%|████████████████████████████████████████████████████████████▎    | 1740/1875 [00:55<00:03, 34.05it/s, loss=0.0463]


 93%|████████████████████████████████████████████████████████████▎    | 1741/1875 [00:56<00:03, 34.05it/s, loss=0.0656]


 93%|████████████████████████████████████████████████████████████▍    | 1742/1875 [00:56<00:03, 35.15it/s, loss=0.0656]


 93%|████████████████████████████████████████████████████████████▍    | 1742/1875 [00:56<00:03, 35.15it/s, loss=0.0709]


 93%|█████████████████████████████████████████████████████████████▎    | 1743/1875 [00:56<00:03, 35.15it/s, loss=0.182]


 93%|█████████████████████████████████████████████████████████████▍    | 1744/1875 [00:56<00:03, 35.15it/s, loss=0.157]


 93%|█████████████████████████████████████████████████████████████▍    | 1745/1875 [00:56<00:03, 35.15it/s, loss=0.119]


 93%|█████████████████████████████████████████████████████████████▍    | 1746/1875 [00:56<00:03, 36.25it/s, loss=0.119]


 93%|███████████████████

 96%|████████████████████████████████████████████████████████████████   | 1794/1875 [00:57<00:02, 37.88it/s, loss=0.12]


 96%|██████████████████████████████████████████████████████████████▏  | 1794/1875 [00:57<00:02, 37.88it/s, loss=0.0884]


 96%|██████████████████████████████████████████████████████████████▏  | 1795/1875 [00:57<00:02, 37.88it/s, loss=0.0886]


 96%|██████████████████████████████████████████████████████████████▎  | 1796/1875 [00:57<00:02, 37.88it/s, loss=0.0796]


 96%|██████████████████████████████████████████████████████████████▎  | 1797/1875 [00:57<00:02, 37.88it/s, loss=0.0788]


 96%|██████████████████████████████████████████████████████████████▎  | 1798/1875 [00:57<00:02, 37.17it/s, loss=0.0788]


 96%|███████████████████████████████████████████████████████████████▎  | 1798/1875 [00:57<00:02, 37.17it/s, loss=0.184]


 96%|███████████████████████████████████████████████████████████████▎  | 1799/1875 [00:57<00:02, 37.17it/s, loss=0.148]


 96%|███████████████████

 99%|████████████████████████████████████████████████████████████████ | 1847/1875 [00:58<00:00, 38.48it/s, loss=0.0911]


 99%|█████████████████████████████████████████████████████████████████ | 1848/1875 [00:58<00:00, 38.48it/s, loss=0.106]


 99%|█████████████████████████████████████████████████████████████████ | 1849/1875 [00:58<00:00, 38.48it/s, loss=0.136]


 99%|█████████████████████████████████████████████████████████████████ | 1850/1875 [00:58<00:00, 37.05it/s, loss=0.136]


 99%|████████████████████████████████████████████████████████████████▏| 1850/1875 [00:58<00:00, 37.05it/s, loss=0.0517]


 99%|████████████████████████████████████████████████████████████████▏| 1851/1875 [00:58<00:00, 37.05it/s, loss=0.0802]


 99%|█████████████████████████████████████████████████████████████████▏| 1852/1875 [00:58<00:00, 37.05it/s, loss=0.231]


 99%|████████████████████████████████████████████████████████████████▏| 1853/1875 [00:58<00:00, 37.05it/s, loss=0.0405]


 99%|███████████████████

The test acc is: 0.91803
